In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

MODEL_H5_PATH='../models/dogs_cats_10_0.687.h5'
MODEL_LITE_PATH='../models/dogs_cats_10_0.687.lite'

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
model = keras.models.load_model(MODEL_H5_PATH)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
lite_model = converter.convert()
with open(MODEL_LITE_PATH, 'wb') as f:
    f.write(lite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path=MODEL_LITE_PATH)
interpreter.allocate_tensors()
output_details = interpreter.get_output_details()
print(output_details)

In [ ]:
img = prepare_image(
    download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'), 
    (150, 150)
)
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

print(X[0][0][0][0])

In [ ]:
# Preprocess Image
img = prepare_image(
    download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'), 
    (150, 150)
)
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=MODEL_LITE_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], X)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)